In [2]:
import numpy as np
import math
import os.path
import csv
import glob
import tensorflow as tf
import h5py as h5py
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model

Using TensorFlow backend.


In [3]:
import imageio
import matplotlib.pyplot as plt
from mlxtend.image import extract_face_landmarks

In [0]:
#
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear
def mouth_aspect_ratio(mouth):
	A = distance.euclidean(mouth[14], mouth[18])
	C = distance.euclidean(mouth[12], mouth[16])
	mar = (A ) / (C)
	return mar

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def circularity(eye):
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area /(p**2)

In [0]:
def mouth_over_eye(eye):
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    mouth_eye = mar/ear
    return mouth_eye

In [0]:
#Extracting Frames from the videos 
from scipy.spatial import distance
import cv2
data = []
labels = []
for j in [18]:
  for i in [10]:
    vidcap = cv2.VideoCapture('/content/drive/My Drive/18/10.mov')
    sec = 0
    frameRate = 1
    success, image  = vidcap.read()
    # print(success)
    count = 0
    while success and count < 240: 
          landmarks = extract_face_landmarks(image)
          # print(landmarks)
          if sum(sum(landmarks)) != 0:
              # print("test")
              count += 1
              data.append(landmarks)
              labels.append([i])
              sec = sec + frameRate
              sec = round(sec, 2)
              success, image = vidcap.read()
              print(count)
          else:  
              sec = sec + frameRate
              sec = round(sec, 2)
              success, image = vidcap.read()
              print("not detected")
        

In [0]:
import numpy as np
data = np.array(data)
labels = np.array(labels)

In [0]:
features = []
label = int(labels[0])
for d in data:
  eye = d[36:68]
  ear = eye_aspect_ratio(eye)
  mar = mouth_aspect_ratio(eye)
  cir = circularity(eye)
  mouth_eye = mouth_over_eye(eye)
  features.append([ear, mar, cir, mouth_eye, label])

In [0]:
features = np.array(features)
features.shape

(240, 5)

In [0]:
#data
features[0]

array([ 0.17231138,  0.98755435,  0.32402916,  5.73121955, 10.        ])

In [0]:
data.shape

(240, 68, 2)

In [0]:
labels.shape

(240, 1)

In [0]:
# change file name to save 

np.savetxt("Fold2_part1_features_label_18_10.csv", features, delimiter = ",")


Merging the data


In [0]:
import pandas as pd 
import glob
path = r'/content/drive/My Drive/Output' # use your path
all_files = glob.glob(path + "/*.csv")

In [10]:
all_files

['/content/drive/My Drive/Output/Fold5_part2_features_label_60_10.csv',
 '/content/drive/My Drive/Output/Fold5_part2_features_label_60_0.csv',
 '/content/drive/My Drive/Output/Fold5_part2_features_label_55_0.csv',
 '/content/drive/My Drive/Output/Fold5_part2_features_label_55_10.csv',
 '/content/drive/My Drive/Output/Fold5_part1_features_label_51_0.csv',
 '/content/drive/My Drive/Output/Fold5_part1_features_label_51_10.csv',
 '/content/drive/My Drive/Output/Fold5_part1_features_label_54_10.csv',
 '/content/drive/My Drive/Output/Fold5_part1_features_label_54_0.csv',
 '/content/drive/My Drive/Output/Fold2_part1_features_label_15_0.csv',
 '/content/drive/My Drive/Output/Fold2_part1_features_label_15_10.csv',
 '/content/drive/My Drive/Output/Fold2_part1_features_label_16_0.csv',
 '/content/drive/My Drive/Output/Fold2_part1_features_label_16_10.csv',
 '/content/drive/My Drive/Output/Fold2_part1_features_label_17_0.csv',
 '/content/drive/My Drive/Output/Fold2_part1_features_label_17_10.csv',

In [0]:
df_total = []
# i,j = 15,0
# df_total = pd.DataFrame()
# for i in [15, 16, 17, 18, 51, 54, 55, 60]:
#   for j in [0, 10]:
#     path = '/Output/Fold2_part1_features_label_'+str(i)+'_'+str(j)+'.csv'
#     df = pd.read_csv(path)
#     df_total.append(df) 

for file in all_files:
  # print(file)
  df = pd.read_csv(file, header= None)
  df_total.append(df)
df_total = pd.concat(df_total)

In [0]:
# Adding Header
df_total.columns = ["EAR", "MAR", "CIR", "Mouth_EYE", "label"]

In [0]:
df_total = df_total.reset_index(drop = True)

In [14]:
df_total.tail()

,EAR,MAR,CIR,Mouth_EYE,label
3835,0.155557,1.010112,0.313427,6.493502,10.0
3836,0.152062,1.018089,0.297301,6.695213,10.0
3837,0.158283,1.010600,0.323855,6.384763,10.0
3838,0.152245,1.045366,0.307941,6.866363,10.0
3839,0.142766,0.978632,0.299788,6.854790,10.0


In [0]:
#Saving final dataframe
df_total.to_csv('total_frames.csv', index = False)